In [19]:
%cd '/content/drive/MyDrive/face_mask_detection/'

/content/drive/MyDrive/face_mask_detection


In [20]:
!unzip 'Dataset.zip' -d '/content/'

Archive:  Dataset.zip
replace /content/Dataset/test/without_mask/0.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: All
  inflating: /content/Dataset/test/without_mask/0.jpg  
  inflating: /content/Dataset/test/without_mask/10.jpg  
  inflating: /content/Dataset/test/without_mask/11.jpg  
  inflating: /content/Dataset/test/without_mask/116.jpg  
  inflating: /content/Dataset/test/without_mask/117.jpg  
  inflating: /content/Dataset/test/without_mask/122.jpg  
  inflating: /content/Dataset/test/without_mask/129.jpg  
  inflating: /content/Dataset/test/without_mask/133.jpg  
  inflating: /content/Dataset/test/without_mask/134.jpg  
  inflating: /content/Dataset/test/without_mask/143.jpg  
  inflating: /content/Dataset/test/without_mask/148.jpg  
  inflating: /content/Dataset/test/without_mask/159.jpg  
  inflating: /content/Dataset/test/without_mask/161.jpg  
  inflating: /content/Dataset/test/without_mask/166.jpg  
  inflating: /content/Dataset/test/without_mask/168.jpg  
  inflating: /conten

In [22]:
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
import cv2
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Conv2D, Input, ZeroPadding2D, BatchNormalization, Activation, MaxPooling2D, Flatten, Dense,Dropout
from keras.models import Model, load_model
from keras.callbacks import TensorBoard, ModelCheckpoint
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.utils import shuffle
import imutils
import numpy as np

##**Train**

In [34]:
model =Sequential([
    Conv2D(100, (3,3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(2,2),
    
    Conv2D(100, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    
    Flatten(),
    Dropout(0.5),
    Dense(50, activation='relu'),
    Dense(2, activation='softmax')
])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

TRAINING_DIR = "/content/Dataset/train"
train_datagen = ImageDataGenerator(rescale=1.0/255,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest')

train_generator = train_datagen.flow_from_directory(TRAINING_DIR,
                                                    batch_size=10,
                                                    target_size=(150, 150))
VALIDATION_DIR = "/content/Dataset/test"
validation_datagen = ImageDataGenerator(rescale=1.0/255)

validation_generator = validation_datagen.flow_from_directory(VALIDATION_DIR,
                                                         batch_size=10,
                                                         target_size=(150, 150))
checkpoint = ModelCheckpoint('models/',monitor='val_loss',verbose=0,save_best_only=True,mode='auto')


history = model.fit_generator(train_generator,
                              epochs=10,
                              validation_data=validation_generator,
                              callbacks=[checkpoint])

Found 1425 images belonging to 2 classes.
Found 392 images belonging to 2 classes.


/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1915: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
143/143 [==============================] - 15s 100ms/step - loss: 0.6984 - acc: 0.6542 - val_loss: 0.2856 - val_acc: 0.8673
INFO:tensorflow:Assets written to: models/assets
Epoch 2/10
143/143 [==============================] - 14s 99ms/step - loss: 0.3644 - acc: 0.8439 - val_loss: 0.5484 - val_acc: 0.7577
Epoch 3/10
143/143 [==============================] - 14s 99ms/step - loss: 0.2873 - acc: 0.8777 - val_loss: 0.4500 - val_acc: 0.7959
Epoch 4/10
143/143 [==============================] - 14s 99ms/step - loss: 0.3078 - acc: 0.8770 - val_loss: 0.4429 - val_acc: 0.8010
Epoch 5/10
143/143 [==============================] - 14s 99ms/step - loss: 0.2487 - acc: 0.9017 - val_loss: 0.4208 - val_acc: 0.8010
Epoch 6/10
143/143 [==============================] - 14s 99ms/step - loss: 0.2637 - acc: 0.8924 - val_loss: 0.4578 - val_acc: 0.7934
Epoch 7/10
143/143 [==============================] - 14s 98ms/step - loss: 0.2275 - acc: 0.9216 - val_loss: 0.4325 - val_acc: 0.8061
Epoch 8/10
1

##**Test**

In [35]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename

In [ ]:
import cv2
import numpy as np
from keras.models import load_model
model=load_model("models/")

labels_dict={1:'without mask',0:'mask'}
color_dict={1:(255,0,),0:(0,255,0)}

size = 4
webcam = cv2.VideoCapture(0) #Use camera 0

# We load the xml file
classifier = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

while True:
    filename = take_photo()
    # (rval, im) = webcam.read()
    # im=cv2.flip(im,1,1) #Flip to act as a mirror
    im = plt.imread(filename)
    # Resize the image to speed up detection
    mini = cv2.resize(im, (im.shape[1] // size, im.shape[0] // size))

    # detect MultiScale / faces 
    faces = classifier.detectMultiScale(mini)

    # Draw rectangles around each face
    for f in faces:
        (x, y, w, h) = [v * size for v in f] #Scale the shapesize backup
        #Save just the rectangle faces in SubRecFaces
        face_img = im[y:y+h, x:x+w]
        resized=cv2.resize(face_img,(150,150))
        normalized=resized/255.0
        reshaped=np.reshape(normalized,(1,150,150,3))
        reshaped = np.vstack([reshaped])
        result=model.predict(reshaped)
        
        label=np.argmax(result,axis=1)[0]
      
        cv2.rectangle(im,(x,y),(x+w,y+h),color_dict[label],2)
        cv2.rectangle(im,(x,y-40),(x+w,y),color_dict[label],-1)
        cv2.putText(im, labels_dict[label], (x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)
        
    # Show the image
    plt.imshow(im)
    plt.show()
    # cv2.imshow('LIVE',   im)
    key = cv2.waitKey(10)
    # if Esc key is press then break out of the loop 
    if key == 27: #The Esc key
        break
# Stop video
webcam.release()

# Close all started windows
cv2.destroyAllWindows()